In [53]:
#setup
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
browser  = webdriver.Chrome(ChromeDriverManager().install())

/tmp/ipykernel_434660/869138682.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser  = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
import pandas as pd
df = pd.read_csv('tabula.csv')
#delete all columns except date, times, film type, film title
df = df[['Date', 'Film', 'Time', 'Time (2nd show)']]

#convert into dictionary with film as the key with multiple possible times as values
film_dict = {}
for index, row in df.iterrows():
    film = row['Film']
    time = row['Time']
    time2 = row['Time (2nd show)']
    if film not in film_dict:
        film_dict[film] = [time, time2]
    else:
        film_dict[film].append(time)
        film_dict[film].append(time2)
print(film_dict)

In [54]:
import pickle
import os
from time import sleep

browser.get('https://www.warwicksu.com/Controls/AssetUploader/AssetBrowser.aspx?asset_type=event&context_id=4273&file_type=image')
if os.path.exists('cookies.pkl'):
    cookies = pickle.load(open("cookies.pkl", "rb"))
    print(cookies)
    for cookie in cookies:
        try:
            
            browser.add_cookie(cookie)
            print("valid:", cookie.keys)
        except:
            print(cookie.keys)
    browser.refresh()
    sleep(1)
    
else:
    input("Please login then press enter to continue...")
    pickle.dump(browser.get_cookies(), open("cookies.pkl", "wb"))

[{'domain': '.warwicksu.com', 'expiry': 1660062905, 'httpOnly': False, 'name': '_gat', 'path': '/', 'secure': False, 'value': '1'}, {'domain': 'www.warwicksu.com', 'httpOnly': True, 'name': '.AspNet.SharedCookie', 'path': '/', 'secure': True, 'value': 'CfDJ8JVR4eL4etFPoGRhWTTSBtBueyZ7bEdQrO_EtD6uxZgPm8DycupcMWdL4oRUgeGK2X9MOmacwus5IU3AFEInZzsJHaR4_LbnQIkouVTZKcqVolCKyTJBPDmjZXC9o4Knl3K5FfrC7YvTzM0f0lq_28-7L4bnF6yIxM1QbG0HdfAerpb7pGmDiMcnGC-iuWh6KfkCA8PrbEzQQUXSFAxKp6P7Pl2o0XYvHWfk_bMquBzrTGwQ4SLMdO-wdapbmrvjYufGIoLG-XxidCfAFpMxsmH_0X9HA8SltCNKG7WZzZhgO_a3mjpu1LXkKL4yHv9BHXPV8HP2E-kP5GslHzD5ZVZMe7cpxqWSvIZvp8GwMXCpRXKHPtKvV8iOG_sG36zhR0WbxpZZHu9UNUyqLSPZKiDIsU2x0tMr0lUdC7zTeIi3px6uRJ0y9SM5QS0fZKo9lBw3340385tQN2qtojm-Y9KAKLiUKnTfOZxbVZJl-ST5gN6D45OVAegd70MKgHwJbm3yIJ15Nhr1KljHCUkkwiA'}, {'domain': '.warwicksu.com', 'expiry': 1660149245, 'httpOnly': False, 'name': '_gid', 'path': '/', 'secure': False, 'value': 'GA1.2.1706601360.1660062781'}, {'domain': '.warwicksu.com', 'expiry': 16946228

In [ ]:
#browser.get('https://www.warwicksu.com/Controls/AssetUploader/AssetBrowser.aspx?asset_type=event&context_id=4273&file_type=image')
#choose files for id ctl00_cphBodyContent_AssetBrowser_fileAsset


In [56]:
import shutil
import requests
import wget
from selenium.webdriver.common.keys import Keys
global api_key
api_key =  "e1312db930d5093c3c7ea7149c644d46"

#get amd dpwm
def getTmdbImageUrl(film_title):
    global api_key
    film_title = film_title.replace(" ", "+")
    url = "https://api.themoviedb.org/3/search/movie?api_key=" + api_key + "&language=en-US" + "&query=" + film_title
    response = requests.get(url)
    data = response.json()
    if len(data['results']) > 0:
        return f"https://image.tmdb.org/t/p/original{data['results'][0]['backdrop_path']}"
    else:
         return None
    
    #download url
    
def downloadImageUrl(url, filename):
    wget.download(url, filename)
    
#find all elements with class msl_row or msl_altrow
def getTitles():
    browser.get("https://www.warwicksu.com/events/edit/4273/")
    input("select date range of films you want to add images to")
    titles = browser.find_elements(By.CLASS_NAME,'msl_row')
    titles.extend(browser.find_elements(By.CLASS_NAME,'msl_altrow'))
    return list(map(lambda x: ' '.join(x.text.split()[3:-6]), titles))


def downloadImages(titles):
    for i in range(len(titles)):
        url = getTmdbImageUrl(titles[i])
        if url is not None:
            downloadImageUrl(url, os.path.join(os.getcwd(),"images", str(i) + ".jpg"))
        else:
            #copy backup to images folder
            backup = os.path.join(os.getcwd(),"images", "back_up.jpg")
            shutil.copy(backup, os.path.join(os.getcwd(),"images", str(i) + ".jpg"))
            print("Unable to find image for:", i, titles[i])




def uploadImages():
    browser.get('https://www.warwicksu.com/Controls/AssetUploader/AssetBrowser.aspx?asset_type=event&context_id=4273&file_type=image')
    i = 0;
    while os.path.exists(os.path.join(os.getcwd(),"images", str(i) + ".jpg")):
        browser.find_element(By.ID,'ctl00_cphBodyContent_AssetBrowser_fileAsset').send_keys(os.path.join(os.getcwd(),"images", str(i) + ".jpg"))
        browser.find_element(By.ID,"ctl00_cphBodyContent_AssetBrowser_btnUpload").click()
        i += 1



def addToEvent():
    browser.get("https://www.warwicksu.com/events/edit/4273/")
    input("select same range as before")
    events = browser.find_elements(By.CLASS_NAME,'msl_row')
    events.extend(browser.find_elements(By.CLASS_NAME,'msl_altrow'))

    for (i, event) in enumerate(events):
        
        url = event.find_element(By.TAG_NAME,'a').get_attribute('href')
        #go to href from event and open in new tab
        browser.execute_script("window.open('');")
        browser.switch_to.window(browser.window_handles[1])
        
        browser.get(url)
        time.sleep(0.1)
        browser.find_element(By.ID,'ctl00_Main_saSelectImage_txtFileName').clear()
        browser.find_element(By.ID,'ctl00_Main_saSelectImage_txtFileName').send_keys(str(i) + ".jpg")

        #press end key on keyboard
        browser.find_element(By.ID,"ctl00_Main_fsSaveEvent_btnSubmit").send_keys(Keys.END)

        time.sleep(0.3)
        browser.find_element(By.ID,"ctl00_Main_fsSaveEvent_btnSubmit").click()

        #close tab and go back to original
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        time.sleep(0.5)


    
    

titles = getTitles()
downloadImages(titles)
uploadImages()
addToEvent()



In [30]:
#event.find_element(By.TAG_NAME,'a').click()
#enter text into ctl00_Main_saSelectImage_txtFileName
i = 0
browser.find_element(By.ID,'ctl00_Main_saSelectImage_txtFileName').send_keys(str(i) + ".jpg")
#scroll down to bottom of page
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(0.5)
browser.find_element(By.ID,"ctl00_Main_fsSaveEvent_btnSubmit").click()